In [79]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.datasets import mnist

In [80]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [114]:
x_train4D = x_train.reshape(60000, 28, 28, 1).astype('float32')
x_test4D = x_test.reshape(10000, 28, 28, 1).astype('float32')

In [116]:
x_train4D_normalize = x_train4D / 255
x_test4D_normalize = x_test4D / 255

In [117]:
# 以 Onehot Encoding 轉換 label
y_train_onehot = tf.keras.utils.to_categorical(y_train)
y_test_onehot = tf.keras.utils.to_categorical(y_test)

In [118]:
# 建立CNN model

# 建立卷積層1
# 輸入數字影像 28x28 的大小，執行一次卷積運算，產生 16 個影像，卷積運算不會改變影像大小，結果還是 28x28
model = tf.keras.Sequential()
model.add(layers.Conv2D(filters=16,
                         kernel_size=(5, 5),
                         padding='same',          # 讓卷積運算產生的影像大小不變
                         input_shape=(28, 28, 1), # input_shape(長, 寬, channel)
                         activation='relu')
)

# 建立池化層
# 輸入參數 pool_size=(2, 2)，執行第一次縮減取樣，將 16 個 28x28 影像，縮小為 16 個 14x14 的影像
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 建立卷積層2
# 執行第二次卷積運算，將原本的 16 個影像，轉換為 36 個影像，卷積運算不會改變影像大小，結果還是 14x14
model.add(layers.Conv2D(filters=36,
                        kernel_size=(5, 5),
                        padding='same',
                        activation='relu')
)

# 建立池化層2，並加入Dropout 避免 overfitting
# 執行第二次縮減取樣，將 36 個 14x14 的影像，縮小為 36 個 7x7 的影像
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

# 建立神經網路 (平坦層, 隱藏層, 輸出層)
# 建立平坦層
# 根據池化層2 的結果，共36 個 7x7 影像，轉換為 1維向量，長度是 36x7x7=1764，也就是 1764 個 float，正好對應到 1764 個神經元
model.add(layers.Flatten())

# 建立隱藏層，共有 128 個神經元
model.add(layers.Dense(128, activation='relu'))

# 加入 Dropout(0.5)
# 每次訓練迭代時，會隨機在神經網路中，放棄 50% 的神經元，以避免 overfitting
model.add(layers.Dropout(0.5))

# 建立輸出層
# 共 10 個神經元，對應 0~9 共 10 個數字，並使用 softmax 激活函數進行轉換
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 1764)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)             

In [119]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [38]:
train_hisyory = model.fit(x=x_train4D_normalize, y=y_train_onehot, batch_size=300, epochs=20, validation_split=0.2)

Epoch 1/20
160/160 [==============================] - 29s 179ms/step - loss: 0.9591 - accuracy: 0.6921 - val_loss: 0.0977 - val_accuracy: 0.9728
Epoch 2/20
160/160 [==============================] - 31s 193ms/step - loss: 0.1494 - accuracy: 0.9555 - val_loss: 0.0686 - val_accuracy: 0.9799
Epoch 3/20
160/160 [==============================] - 30s 187ms/step - loss: 0.1048 - accuracy: 0.9698 - val_loss: 0.0530 - val_accuracy: 0.9845
Epoch 4/20
160/160 [==============================] - 29s 180ms/step - loss: 0.0874 - accuracy: 0.9743 - val_loss: 0.0467 - val_accuracy: 0.9858
Epoch 5/20
160/160 [==============================] - 30s 187ms/step - loss: 0.0709 - accuracy: 0.9780 - val_loss: 0.0390 - val_accuracy: 0.9894
Epoch 6/20
160/160 [==============================] - 31s 195ms/step - loss: 0.0603 - accuracy: 0.9816 - val_loss: 0.0396 - val_accuracy: 0.9888
Epoch 7/20
160/160 [==============================] - 30s 190ms/step - loss: 0.0536 - accuracy: 0.9839 - val_loss: 0.0353 - val_ac

In [41]:
score = model.evaluate(x_test4D_normalize, y_test_onehot)
print("Accuracy:{}%".format(score[1]))

313/313 [==============================] - 2s 6ms/step - loss: 0.0183 - accuracy: 0.9943
Accuracy:0.9943000078201294%
